In [20]:
# Импортируем необходимые классы и функции из библиотек
from transformers import pipeline, AutoImageProcessor, AutoModelForImageClassification
from datasets import load_dataset
from transformers.pipelines.pt_utils import KeyDataset
from tqdm.auto import tqdm

# Имя модели на HuggingFaceHub.
MODEL_NAME = "panda992/fish_disease_datasets"
# Имя датасета на HuggingFaceHub.
DATASET_NAME = "annassdan/autotrain-data-fish-classification"

def run_fish_disease_classification_pipeline():

    """
    Основная функция для демонстрации загрузки модели, датасета,
    получения предсказаний и вывода результатов.
    """

    # Загружаем датасет
    # Загружаем датасет с HuggingFaceHub. Он разделяется на 'train' и 'validation' (если они есть).
    ds = load_dataset(DATASET_NAME)

    # Для примера берем только тренировочную часть датасета для обработки.
    train_dataset = ds['train']
    print(f"Датасет загружен. Количество изображений в 'train' части: {len(train_dataset)}")

    # Загружаем модель и подготавливаем pipeline
    # Инициализируем pipeline для классификации изображений.
    # device=0 использует GPU, если доступно, для ускорения расчетов. Если GPU нет, используйте device=-1 (CPU).
    pipe = pipeline("image-classification", model=MODEL_NAME, device=0)

    # !!! Показываем альтернативный способ загрузки модели и процессора отдельно (если нужно вручную обработать изображения).
    #     processor = AutoImageProcessor.from_pretrained(MODEL_NAME)
    #     model = AutoModelForImageClassification.from_pretrained(MODEL_NAME) !!!

    # Получаем предсказания
    # Создаем список для хранения результатов предсказаний.
    predictions = []

    # Используем KeyDataset для указания, что нужно брать данные из колонки 'image' в датасете.
    # Это позволяет передавать весь датасет напрямую в pipeline
    # tqdm добавляет индикатор прогресса в цикле.
    for output in tqdm(pipe(KeyDataset(train_dataset, "image")), total=len(train_dataset)):
        predictions.append(output)

In [22]:
# Обработка и вывод результатов
print("\nПредсказания для первых 30 изображений:") # Взял побольше, так как изображений с заращёнными рыбами мало

for i in range(30):
    # Получаем информацию об i-м изображении из датасета
    img_info = train_dataset[i]

    # Модель pipeline возвращает список словарей для каждого изображения.
    # 0 соответствует наиболее вероятному предсказанию.
    best_pred_dict = predictions[i][0]

    # Получаем реальную метку класса из датасета (если колонка 'label' существует, иначе 'Нет')
    true_label = img_info.get('label', 'Нет (метка отсутствует)')

    print(f"Изображение #{i+1}")
    print(f"  Реальный класс (dataset label_id): {true_label}")
    # Теперь обращаемся к элементу словаря по ключу 'label' и 'score'
    print(f"  Предсказание модели (класс): {best_pred_dict['label']}")
    print(f"  Вероятность: {best_pred_dict['score']:.4f}\n")




Предсказания для первых 30 изображений:
Изображение #1
  Реальный класс (dataset label_id): 0
  Предсказание модели (класс): Healthy Fish
  Вероятность: 0.6035

Изображение #2
  Реальный класс (dataset label_id): 1
  Предсказание модели (класс): Parasitic diseases
  Вероятность: 0.5169

Изображение #3
  Реальный класс (dataset label_id): 1
  Предсказание модели (класс): Healthy Fish
  Вероятность: 0.5681

Изображение #4
  Реальный класс (dataset label_id): 1
  Предсказание модели (класс): Healthy Fish
  Вероятность: 0.9702

Изображение #5
  Реальный класс (dataset label_id): 1
  Предсказание модели (класс): Healthy Fish
  Вероятность: 0.6268

Изображение #6
  Реальный класс (dataset label_id): 1
  Предсказание модели (класс): Bacterial Red disease
  Вероятность: 0.7110

Изображение #7
  Реальный класс (dataset label_id): 1
  Предсказание модели (класс): Healthy Fish
  Вероятность: 0.8922

Изображение #8
  Реальный класс (dataset label_id): 1
  Предсказание модели (класс): Healthy Fish